In [17]:
from django.conf import settings
from web3 import Web3
import json
import os

from web3.middleware import geth_poa_middleware

rinkeby_bridge_address = settings.RINKEBY_BRIDGE_CONTRACT_ADDRESS
rinkeby_web3 = Web3(Web3.HTTPProvider(settings.INFURA_RINKEBY_HTTP_URL))
abi = None
with open(settings.RINKEBY_BRIDGE_CONTRACT_ABI_FILEPATH) as f:
    abi = json.loads(f.read())

rinkeby_bridge_contract = rinkeby_web3.eth.contract(address=rinkeby_bridge_address, abi=abi)

goerli_bridge_address = settings.GOERLI_BRIDGE_CONTRACT_ADDRESS
goerli_web3 = Web3(Web3.HTTPProvider(settings.INFURA_GOERLI_HTTP_URL))
goerli_web3.middleware_onion.inject(geth_poa_middleware, layer=0)
abi = None
with open(settings.GOERLI_BRIDGE_CONTRACT_ABI_FILEPATH) as f:
    abi = json.loads(f.read())

goerli_bridge_contract = goerli_web3.eth.contract(address=goerli_bridge_address, abi=abi)


In [18]:
# build transaction
sender = '0x41d34752F32dAbf56367FA7aC7b021562a173854'

args = {
    'to': '0x724beDCD1C201eA956E2389f66F4c73589424826',
    'amount': 10000000000000000000
}
func = rinkeby_bridge_contract.functions.burn(settings.RINKEBY_VCHAIN_TOKEN_CONTRACT_ADDRESS, args['to'], args['amount'])
tx = func.buildTransaction({
#     'gas': 70000,
#     'gasPrice': rinkeby_web3.eth.gas_price,
    'from': sender,
    'nonce': rinkeby_web3.eth.getTransactionCount(sender)
    })

sender_private_key = settings.ETHEREUM_PRIVATE_KEY

# sign transaction
private_key = rinkeby_web3.toHex(hexstr=sender_private_key)
signed_tx = rinkeby_web3.eth.account.sign_transaction(tx, private_key=private_key)

In [19]:
# send transaction

rinkeby_web3.eth.send_raw_transaction(signed_tx.rawTransaction)

HexBytes('0x3858d03f43d94672c04fca199b55f64301e56477bc62a27ffb01c3b89d402912')